# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --mem-fraction-static 0.8"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-05-18 02:06:43] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, host='0.0.0.0', port=35109, mem_fraction_static=0.8, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=548907388, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='in

[2025-05-18 02:06:54] Attention backend not set. Use fa3 backend by default.
[2025-05-18 02:06:54] Init torch distributed begin.


[2025-05-18 02:06:54] Init torch distributed ends. mem usage=0.00 GB
[2025-05-18 02:06:54] Load weight begin. avail mem=77.02 GB


[2025-05-18 02:06:56] Using model weights format ['*.safetensors']


[2025-05-18 02:06:56] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-05-18 02:06:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-05-18 02:06:58] INFO:     Started server process [2294344]
[2025-05-18 02:06:58] INFO:     Waiting for application startup.
[2025-05-18 02:06:58] INFO:     Application startup complete.
[2025-05-18 02:06:58] INFO:     Uvicorn running on http://0.0.0.0:35109 (Press CTRL+C to quit)
[2025-05-18 02:06:58] INFO:     127.0.0.1:35336 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-18 02:06:59] INFO:     127.0.0.1:35342 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-18 02:06:59] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 02:07:01] INFO:     127.0.0.1:35358 - "POST /generate HTTP/1.1" 200 OK
[2025-05-18 02:07:01] The server is fired up and ready to roll!


Server started on http://localhost:35109


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-05-18 02:07:03] Prefill batch. #new-seq: 1, #new-token: 37, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 02:07:04] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.46, #queue-req: 0
[2025-05-18 02:07:04] INFO:     127.0.0.1:53794 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

SGLang extends the standard API with the `extra_body` parameter, allowing for additional customization. One key option within `extra_body` is `chat_template_kwargs`, which can be used to pass arguments to the chat template processor.

#### Enabling Model Thinking/Reasoning

You can use `chat_template_kwargs` to enable or disable the model's internal thinking or reasoning process output. Set `"enable_thinking": True` within `chat_template_kwargs` to include the reasoning steps in the response. This requires launching the server with a compatible reasoning parser (e.g., `--reasoning-parser qwen3` for Qwen3 models).

Here's an example demonstrating how to enable thinking and retrieve the reasoning content separately (using `separate_reasoning: True`):

```python
# Ensure the server is launched with a compatible reasoning parser, e.g.:
# python3 -m sglang.launch_server --model-path QwQ/Qwen3-32B-250415 --reasoning-parser qwen3 ...

from openai import OpenAI

# Modify OpenAI's API key and API base to use SGLang's API server.
openai_api_key = "EMPTY"
openai_api_base = f"http://127.0.0.1:{port}/v1" # Use the correct port

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

model = "QwQ/Qwen3-32B-250415" # Use the model loaded by the server
messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]

response = client.chat.completions.create(
    model=model,
    messages=messages,
    extra_body={
        "chat_template_kwargs": {"enable_thinking": True},
        "separate_reasoning": True
    }
)

print("response.choices[0].message.reasoning_content: \n", response.choices[0].message.reasoning_content)
print("response.choices[0].message.content: \n", response.choices[0].message.content)
```

**Example Output:**

```
response.choices[0].message.reasoning_content: 
 Okay, so I need to figure out which number is greater between 9.11 and 9.8. Hmm, let me think. Both numbers start with 9, right? So the whole number part is the same. That means I need to look at the decimal parts to determine which one is bigger.
...
Therefore, after checking multiple methods—aligning decimals, subtracting, converting to fractions, and using a real-world analogy—it's clear that 9.8 is greater than 9.11.

response.choices[0].message.content: 
 To determine which number is greater between **9.11** and **9.8**, follow these steps:
...
**Answer**:  
9.8 is greater than 9.11.
```

Setting `"enable_thinking": False` (or omitting it) will result in `reasoning_content` being `None`.

Here is an example of a detailed chat completion request using standard OpenAI parameters:

In [3]:
response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-05-18 02:07:04] Prefill batch. #new-seq: 1, #new-token: 49, #cached-token: 5, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 02:07:04] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, cuda graph: False, gen throughput (token/s): 72.94, #queue-req: 0


[2025-05-18 02:07:05] Decode batch. #running-req: 1, #token: 128, token usage: 0.01, cuda graph: False, gen throughput (token/s): 79.04, #queue-req: 0


[2025-05-18 02:07:05] Decode batch. #running-req: 1, #token: 168, token usage: 0.01, cuda graph: False, gen throughput (token/s): 83.63, #queue-req: 0
[2025-05-18 02:07:05] INFO:     127.0.0.1:53794 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-05-18 02:07:06] INFO:     127.0.0.1:53794 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-05-18 02:07:06] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 24, token usage: 0.00, #running-req: 0, #queue-req: 0


Yes, I am indeed a test created by Alibaba Cloud. I am designed

 to provide information

 and assistance on various topics, including[2025-05-18 02:07:06] Decode batch. #running-req: 1, #token: 60, token usage: 0.00, cuda graph: False, gen throughput (token/s): 80.85, #queue-req: 0
 AI, language, and more. My

 purpose is to

 help users with their queries and provide helpful responses. If you have any specific

 questions or topics

 you'd like to discuss, feel free to ask and[2025-05-18 02:07:06] Decode batch. #running-req: 1, #token: 100, token usage: 0.00, cuda graph: False, gen throughput (token/s): 86.22, #queue-req: 0
 I'll do my

 best to help

.

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-05-18 02:07:06] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 02:07:07] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 75.91, #queue-req: 0


[2025-05-18 02:07:07] INFO:     127.0.0.1:53794 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-05-18 02:07:07] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-18 02:07:07] Decode batch. #running-req: 1, #token: 17, token usage: 0.00, cuda graph: False, gen throughput (token/s): 81.43, #queue-req: 0


[2025-05-18 02:07:08] INFO:     127.0.0.1:53794 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structured outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-05-18 02:07:08] INFO:     127.0.0.1:53800 - "POST /v1/files HTTP/1.1" 200 OK
[2025-05-18 02:07:08] INFO:     127.0.0.1:53800 - "POST /v1/batches HTTP/1.1" 200 OK


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-05-18 02:07:08] Decode batch. #running-req: 2, #token: 88, token usage: 0.00, cuda graph: False, gen throughput (token/s): 75.71, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2025-05-18 02:07:11] INFO:     127.0.0.1:53800 - "GET /v1/batches/batch_684439e0-6215-4480-8cf4-6c1fbbacecf1 HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-05-18 02:07:11] INFO:     127.0.0.1:53800 - "GET /v1/files/backend_result_file-ee87be75-5a63-458b-a36c-34c2689d3b4d/content HTTP/1.1" 200 OK


[2025-05-18 02:07:11] INFO:     127.0.0.1:53800 - "DELETE /v1/files/backend_result_file-ee87be75-5a63-458b-a36c-34c2689d3b4d HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-05-18 02:07:11] INFO:     127.0.0.1:38476 - "POST /v1/files HTTP/1.1" 200 OK
[2025-05-18 02:07:11] INFO:     127.0.0.1:38476 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-05-18 02:07:11] Prefill batch. #new-seq: 8, #new-token: 240, #cached-token: 24, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-18 02:07:11] Prefill batch. #new-seq: 12, #new-token: 370, #cached-token: 36, token usage: 0.01, #running-req: 8, #queue-req: 0


[2025-05-18 02:07:11] Decode batch. #running-req: 20, #token: 843, token usage: 0.04, cuda graph: False, gen throughput (token/s): 91.24, #queue-req: 0


[2025-05-18 02:07:11] Decode batch. #running-req: 19, #token: 1561, token usage: 0.08, cuda graph: False, gen throughput (token/s): 1551.73, #queue-req: 0


[2025-05-18 02:07:21] INFO:     127.0.0.1:33968 - "GET /v1/batches/batch_4e688c61-2ccf-47b7-bd4f-dece21dbd098 HTTP/1.1" 200 OK


[2025-05-18 02:07:24] INFO:     127.0.0.1:33968 - "GET /v1/batches/batch_4e688c61-2ccf-47b7-bd4f-dece21dbd098 HTTP/1.1" 200 OK


[2025-05-18 02:07:27] INFO:     127.0.0.1:33968 - "GET /v1/batches/batch_4e688c61-2ccf-47b7-bd4f-dece21dbd098 HTTP/1.1" 200 OK


[2025-05-18 02:07:30] INFO:     127.0.0.1:33968 - "GET /v1/batches/batch_4e688c61-2ccf-47b7-bd4f-dece21dbd098 HTTP/1.1" 200 OK


[2025-05-18 02:07:33] INFO:     127.0.0.1:33968 - "GET /v1/batches/batch_4e688c61-2ccf-47b7-bd4f-dece21dbd098 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-05-18 02:07:36] INFO:     127.0.0.1:42872 - "POST /v1/files HTTP/1.1" 200 OK
[2025-05-18 02:07:36] INFO:     127.0.0.1:42872 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-05-18 02:07:37] Prefill batch. #new-seq: 7, #new-token: 7, #cached-token: 224, token usage: 0.01, #running-req: 0, #queue-req: 0
[2025-05-18 02:07:37] Prefill batch. #new-seq: 124, #new-token: 3343, #cached-token: 901, token usage: 0.03, #running-req: 7, #queue-req: 35


[2025-05-18 02:07:38] Prefill batch. #new-seq: 29, #new-token: 870, #cached-token: 145, token usage: 0.27, #running-req: 130, #queue-req: 4840
[2025-05-18 02:07:38] Prefill batch. #new-seq: 8, #new-token: 240, #cached-token: 40, token usage: 0.32, #running-req: 158, #queue-req: 4832


[2025-05-18 02:07:38] Decode batch. #running-req: 165, #token: 9098, token usage: 0.44, cuda graph: False, gen throughput (token/s): 160.02, #queue-req: 4832


[2025-05-18 02:07:39] Decode batch. #running-req: 157, #token: 14967, token usage: 0.73, cuda graph: False, gen throughput (token/s): 10977.56, #queue-req: 4832


...

[2025-05-18 02:07:45] Decode batch. #running-req: 133, #token: 14530, token usage: 0.71, cuda graph: False, gen throughput (token/s): 8714.48, #queue-req: 4466


[2025-05-18 02:07:46] Decode batch. #running-req: 133, #token: 18436, token usage: 0.90, cuda graph: False, gen throughput (token/s): 9051.10, #queue-req: 4466
[2025-05-18 02:07:46] Prefill batch. #new-seq: 6, #new-token: 180, #cached-token: 30, token usage: 0.91, #running-req: 124, #queue-req: 4460
[2025-05-18 02:07:46] Prefill batch. #new-seq: 9, #new-token: 282, #cached-token: 33, token usage: 0.86, #running-req: 121, #queue-req: 4451
[2025-05-18 02:07:46] Prefill batch. #new-seq: 100, #new-token: 3102, #cached-token: 398, token usage: 0.13, #running-req: 30, #queue-req: 4351


[2025-05-18 02:07:46] Prefill batch. #new-seq: 14, #new-token: 430, #cached-token: 60, token usage: 0.30, #running-req: 128, #queue-req: 4337
[2025-05-18 02:07:46] Prefill batch. #new-seq: 8, #new-token: 248, #cached-token: 32, token usage: 0.32, #running-req: 135, #queue-req: 4329
[2025-05-18 02:07:46] INFO:     127.0.0.1:34100 - "POST /v1/batches/batch_36c297e3-6a73-4c2a-b0e4-430aa555aa2b/cancel HTTP/1.1" 200 OK


[2025-05-18 02:07:49] INFO:     127.0.0.1:34100 - "GET /v1/batches/batch_36c297e3-6a73-4c2a-b0e4-430aa555aa2b HTTP/1.1" 200 OK


[2025-05-18 02:07:49] INFO:     127.0.0.1:34100 - "DELETE /v1/files/backend_input_file-649d8079-e02e-41b9-8eb0-4ca9d5c51389 HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)

[2025-05-18 02:07:49] Child process unexpectedly failed with an exit code 9. pid=2294904
